In [28]:
import cv2 as cv 
import numpy as np
import pandas as pd

import os
import glob

In [ ]:
X = []
Y = []
DIR = r"E:\Dataset-2"

classifier = cv.CascadeClassifier(cv.data.haarcascades + "haarcascade_frontalface_default.xml")

In [ ]:
for x in os.listdir(DIR):
    folder_path = os.path.join(DIR, x)
    if not os.path.isdir(folder_path):
        continue

    video_list = glob.glob(os.path.join(folder_path, '*.avi'))
    txt_list = glob.glob(os.path.join(folder_path, '*.txt'))

    if not video_list or not txt_list:
        continue

    video = video_list[0]
    txt = txt_list[0]

    cap = cv.VideoCapture(video)
    with open('E:\Dataset-2\subject8\ground_truth.txt', 'r') as f:
        lines = f.readlines()

    hr_values = list(map(float, lines[1].strip().split()))
    time_values = list(map(float, lines[2].strip().split()))

    fps = cap.get(cv.CAP_PROP_FPS)
    
    X_feature = []
    Y_feature = []

    total_rows = len(time_values)
    usable_rows = total_rows - (total_rows % 24)

    for i in range(usable_rows):
        timestamp_ms = time_values[i]
        frame_number = int((timestamp_ms) * fps)
        cap.set(cv.CAP_PROP_POS_FRAMES, frame_number)

        ret, frame = cap.read()
        if not ret:
            break

        gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
        green = frame[:, :, 1]

        faces = classifier.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=3)
        if len(faces) == 0:
            continue

        x, y, w, h = faces[0]
        face_green = green[y:y+h, x:x+w]
        mean_g = np.mean(face_green)

        X_feature.append([mean_g])
        Y_feature.append([hr_values[i]])

    b = int(len(X_feature) / 24)
    X_feature = np.array(X_feature[:b*24]).reshape(b, 24, 1)
    Y_feature = np.array(Y_feature[:b*24]).reshape(b, 24)

    X.append(X_feature)
    Y.append(Y_feature)


In [ ]:
# import mediapipe as mp

# DIR = r"E:\Dataset-2"

# mp_face_detection = mp.solutions.face_detection
# mp_drawing = mp.solutions.drawing_utils

# X, Y = [], []

# with mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence=0.5) as face_detection:
#     for x in os.listdir(DIR):
#         folder_path = os.path.join(DIR, x)
#         if not os.path.isdir(folder_path):
#             continue

#         video_list = glob.glob(os.path.join(folder_path, '*.avi'))
#         txt_list = glob.glob(os.path.join(folder_path, '*.txt'))

#         if not video_list or not txt_list:
#             continue

#         video = video_list[0]
#         txt = txt_list[0]

#         cap = cv.VideoCapture(video)
#         with open(txt, 'r') as f:
#             lines = f.readlines()

#         hr_values = list(map(float, lines[1].strip().split()))
#         time_values = list(map(float, lines[2].strip().split()))

#         fps = cap.get(cv.CAP_PROP_FPS)

#         X_feature = []
#         Y_feature = []

#         total_rows = len(time_values)
#         usable_rows = total_rows - (total_rows % 24)

#         for i in range(usable_rows):
#             timestamp_ms = time_values[i]
#             frame_number = int(timestamp_ms * fps)
#             cap.set(cv.CAP_PROP_POS_FRAMES, frame_number)

#             ret, frame = cap.read()
#             if not ret:
#                 break

#             rgb_frame = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
#             results = face_detection.process(rgb_frame)

#             if not results.detections:
#                 continue

#             # Get first detected face
#             detection = results.detections[0]
#             bboxC = detection.location_data.relative_bounding_box

#             ih, iw, _ = frame.shape
#             x1 = int(bboxC.xmin * iw)
#             y1 = int(bboxC.ymin * ih)
#             w = int(bboxC.width * iw)
#             h = int(bboxC.height * ih)

#             # Ensure the coordinates are within bounds
#             x1, y1 = max(0, x1), max(0, y1)
#             x2, y2 = min(iw, x1 + w), min(ih, y1 + h)

#             face_green = frame[y1:y2, x1:x2, 1]  # Green channel only
#             mean_g = np.mean(face_green)

#             X_feature.append([mean_g])
#             Y_feature.append([hr_values[i]])

#         b = int(len(X_feature) / 24)
#         X_feature = np.array(X_feature[:b*24]).reshape(b, 24, 1)
#         Y_feature = np.array(Y_feature[:b*24]).reshape(b, 24)

#         X.append(X_feature)
#         Y.append(Y_feature)


In [ ]:
X

[array([[[129.51707451],
         [129.75986121],
         [129.75986121],
         ...,
         [130.60593834],
         [130.00177458],
         [130.01250866]],
 
        [[129.98844356],
         [129.97351108],
         [130.45717293],
         ...,
         [128.9816    ],
         [128.2174466 ],
         [127.99740305]],
 
        [[127.97649758],
         [128.05176593],
         [128.16763331],
         ...,
         [128.49357484],
         [127.99792244],
         [128.56791369]],
 
        ...,
 
        [[131.97293113],
         [131.50774066],
         [131.78411415],
         ...,
         [134.20871436],
         [134.42070026],
         [133.78321678]],
 
        [[132.88565993],
         [132.79828775],
         [132.72413793],
         ...,
         [132.38305363],
         [131.76489893],
         [131.74929845]],
 
        [[131.75082045],
         [131.69850178],
         [131.6508918 ],
         ...,
         [131.47082004],
         [131.8224351 ],
         [1

In [ ]:
X = np.vstack(X)  # or np.concatenate(X, axis=0) depending on shape
Y = np.vstack(Y)

np.save("input_media.npy", X)
np.save("output_media.npy", Y)

In [ ]:
out=np.mean(Y,axis=1).astype("int64")
out=out.reshape(-1,1)

In [ ]:
print(len(Y))
print(len(X))

2774
2774


In [ ]:
X

array([[[129.51707451],
        [129.75986121],
        [129.75986121],
        ...,
        [130.60593834],
        [130.00177458],
        [130.01250866]],

       [[129.98844356],
        [129.97351108],
        [130.45717293],
        ...,
        [128.9816    ],
        [128.2174466 ],
        [127.99740305]],

       [[127.97649758],
        [128.05176593],
        [128.16763331],
        ...,
        [128.49357484],
        [127.99792244],
        [128.56791369]],

       ...,

       [[133.57136481],
        [133.65592144],
        [133.69455465],
        ...,
        [133.44390994],
        [133.38667923],
        [133.38667923]],

       [[133.27886332],
        [133.27886332],
        [133.25902599],
        ...,
        [132.94673306],
        [133.12854592],
        [133.11619718]],

       [[133.12398334],
        [133.09249157],
        [133.2437772 ],
        ...,
        [133.03329812],
        [132.98849435],
        [132.79679091]]])

In [ ]:
Y

array([[ 98. ,  98. ,  98. , ...,  98. ,  98. ,  98. ],
       [ 98. ,  98. ,  98. , ...,  98. ,  98. ,  98. ],
       [ 98. ,  98. ,  98. , ...,  98. ,  98. ,  98. ],
       ...,
       [102. , 102. , 102. , ..., 101. , 101. , 101. ],
       [101. , 101. , 101. , ..., 101. , 100.4, 100. ],
       [100. , 100. , 100. , ..., 100. , 100. , 100. ]])

In [ ]:
out

array([[ 97],
       [ 97],
       [ 97],
       [ 97],
       [ 97],
       [ 98],
       [ 99],
       [101],
       [103],
       [106],
       [107],
       [109],
       [110],
       [110],
       [111],
       [113],
       [113],
       [115],
       [116],
       [117],
       [117],
       [117],
       [115],
       [114],
       [112],
       [111],
       [111],
       [111],
       [112],
       [113],
       [115],
       [115],
       [115],
       [115],
       [115],
       [115],
       [115],
       [115],
       [115],
       [114],
       [113],
       [112],
       [111],
       [111],
       [111],
       [112],
       [111],
       [111],
       [110],
       [110],
       [109],
       [109],
       [110],
       [111],
       [111],
       [113],
       [114],
       [115],
       [115],
       [115],
       [114],
       [113],
       [112],
       [111],
       [110],
       [110],
       [110],
       [111],
       [112],
       [113],
       [113],
      

In [ ]:
np.save("out_media.npy", out)

In [ ]:
print(len(X))
print(len(Y))
print(len(out))

2774
2774
2774


In [ ]:
Y.shape

(2774, 24)

In [ ]:
out.shape

(2774, 1)

In [ ]:
out

array([[ 98],
       [ 98],
       [ 98],
       ...,
       [101],
       [100],
       [100]], dtype=int64)